# Mask R-CNN - Inspect Ballon Trained Model

Code and visualizations to test, debug, and evaluate the Mask R-CNN model.

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage import io

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
import mrcnn.model as modellib
from mrcnn.model import log

from samples.japan_roof import japan_roof
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Path to Ballon trained weights
# You can download this file from the Releases page
# https://github.com/matterport/Mask_RCNN/releases

ckpt_number = 9245
BALLON_WEIGHTS_PATH = "../../logs/pascalvoc20200629T0141/mask_rcnn_pascalvoc_" + str(ckpt_number) + ".h5"  # TODO: update this path

## Configurations

In [ ]:
config = japan_roof.PascalVOCConfig()
JAPAN_ROOF_DIR = os.path.join(ROOT_DIR, "/root/japan_roof_dataset_2000_solarpanel")
print(JAPAN_ROOF_DIR)

In [ ]:
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

## Notebook Preferences

In [ ]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

In [ ]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Load Validation Dataset

In [ ]:
# Load validation dataset
dataset = japan_roof.PascalVOCDataset()
dataset.load_pascalvoc(JAPAN_ROOF_DIR, "val")

# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

## Load Model

In [ ]:
# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

In [ ]:
# Set path to balloon weights file

# Download file from the Releases page and set its path
# https://github.com/matterport/Mask_RCNN/releases
# weights_path = "/path/to/mask_rcnn_balloon.h5"

# Or, load the last model you trained
weights_path = model.find_last()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

## Run Detection

In [ ]:
import colorsys

def random_colors(N, bright=True):
    """
    Generate random colors.
    To get visually distinct colors, generate them in HSV space then
    convert to RGB.
    """
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.shuffle(colors)
    return colors


In [ ]:
# 면적 계산을 위한 행렬의 합을 계산

def solution(arr1, arr2):
    return [[c + d for c, d in zip(a, b)] for a, b in zip(arr1,arr2)]

In [ ]:
def apply_mask(image, mask, color, alpha=0.5):
    """Apply the given mask to the image.
    """
    for c in range(3):
        image[:, :, c] = np.where(mask == 1,
                                  image[:, :, c] *
                                  (1 - alpha) + alpha * color[c],
                                  image[:, :, c])
    return image

In [ ]:
import cv2
    
result_save_dir = "./inference_result_" + str(ckpt_number) + "/"
if not os.path.exists(result_save_dir):
    os.mkdir(result_save_dir)

cnt = 0
# for path, dirs, files in os.walk("/root/japan_roof_dataset_2000_solarpanel/testtest/"):
for path, dirs, files in os.walk("/root/japan_roof_with_trees/"):
    for file in files:
        cnt += 1
        image_path = os.path.join(path, file)
        image = io.imread(image_path)

        # png 이미지에 alpha 채널이 있다면 제거 (640, 640 ,4)  >> (640, 640, 3)
        if image.shape[-1] == 4:
            image = image[..., :3]
        
        results = model.detect([image], verbose=1)
        
        # Display results
        r = results[0]
        
        boxes = r['rois']
        masks = r['masks']
        class_ids = r['class_ids']
        class_names = dataset.class_names
        scores = r['scores']
        show_bbox = True
        show_mask = True
        captions = None
        
        # Number of instances
        N = boxes.shape[0]
        print('number_of_instances : ', N)
        if not N:
            print("\n*** No instances to display *** \n")
        else:
            assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]

        # Generate random colors
        colors = random_colors(N)

        # Show area outside image boundaries.
        height, width = image.shape[:2]

        masked_image = image.astype(np.uint32).copy()
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # 이미지 사이즈의 Flase array 준비
        flat_mask = np.full((640,640), False, dtype=bool)
        trees_mask = np.full((640,640), False, dtype=bool)
        solarpanel_mask = np.full((640,640), False, dtype=bool)
        
        for i in range(N):
            float_color = colors[i]
            color = [int(element * 255) for element in float_color]
            
            # Bounding box
            if not np.any(boxes[i]):
                # Skip this instance. Has no bbox. Likely lost in image cropping.
                continue
            y1, x1, y2, x2 = boxes[i]

            if show_bbox:
                cv2.rectangle(image, (x1, y1), (x2, y2), color, 1)
                
                
            # Label
            if not captions:
                class_id = class_ids[i]
                score = scores[i] if scores is not None else None
                label = class_names[class_id]
                caption = "{} {:.3f}".format(label, score) if score else label
            else:
                caption = captions[i]
            cv2.putText(image, caption, (x1, y1 + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            
            
            # Mask
            mask = masks[:, :, i]
            if show_mask:
                masked_image = apply_mask(image, mask, color)
            
            if label == 'goodroof':
                flat_mask = solution(flat_mask, mask)
                
            if label == 'trees':
                trees_mask = solution(trees_mask, mask)

            if label == 'solarpanel':
                solarpanel_mask = solution(solarpanel_mask, mask)
            
#             print("label : ", label)
#             print("instance mask size : ", np.count_nonzero(mask))
#             print("flat_mask : ", np.count_nonzero(flat_mask))
            
            # 한 이미지의 마지막 루프 때 면적 계산결과를 표시
            if i == N - 1:
                flat_text = "Ratio of Flat : " + str("%0.2f"%(np.count_nonzero(flat_mask)/409600)) # 640*640 = 409600
                solarpanel_text = "Ratio of solarpanel : " + str("%0.2f"%(np.count_nonzero(solarpanel_mask)/409600)) # 640*640 = 409600
                trees_text = "Ratio of trees : " + str("%0.2f"%(np.count_nonzero(trees_mask)/409600)) # 640*640 = 409600
                
                cv2.rectangle(masked_image, (380,580), (640,640), (255, 255, 255), (-1))
                cv2.putText(masked_image, flat_text, (380, 580 + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
                cv2.putText(masked_image, solarpanel_text, (380, 580 + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
                cv2.putText(masked_image, trees_text, (380, 580 + 45), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)


        cv2.imwrite(result_save_dir + file, masked_image)
        print("Image count : ", cnt)
        
        
        

In [ ]:
flat_mask = []
mask1 = np.full((3,4), False, dtype=bool)

mask2 = [
    [False, False, False, False],
    [True, False, False, True],
    [False, False, False, False]
    
]

print(mask1)
mask1 = solution(mask1, mask2)
print(mask1)